# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fort st. john,56.2499,-120.8529,4.94,100,100,3.60,CA,1697312627
1,1,dudinka,69.4058,86.1778,1.49,93,100,6.71,RU,1697312439
2,2,waitangi,-43.9535,-176.5597,13.79,88,1,4.92,NZ,1697312449
3,3,huayucachi,-12.1333,-75.2333,17.49,48,100,4.48,PE,1697312801
4,4,port-aux-francais,-49.3500,70.2167,1.13,68,64,9.60,TF,1697312449


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM")

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] <= 27)
                                       & (city_data_df["Max Temp"] >= 21) 
                                       & (city_data_df["Cloudiness"] == 0)
                                       & (city_data_df["Wind Speed"] < 4.5)]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,115,gwadar,25.1216,62.3254,26.60,85,0,4.18,PK,1697312824
119,119,gremda,34.7500,10.7833,25.11,65,0,0.00,TN,1697312824
140,140,pacific grove,36.6177,-121.9166,25.23,62,0,2.57,US,1697312665
282,282,birzebbuga,35.8258,14.5269,22.86,78,0,2.06,MT,1697312858
292,292,paulden,34.8856,-112.4682,26.42,30,0,1.79,US,1697312860
329,329,metlili chaamba,32.2667,3.6333,25.52,21,0,2.57,DZ,1697312866
375,375,mckinleyville,40.9465,-124.1006,22.66,79,0,2.06,US,1697312875
396,396,laguna,38.4210,-121.4238,26.67,53,0,2.57,US,1697312750
419,419,teguise,29.0605,-13.5640,24.12,61,0,2.06,ES,1697312884
559,559,san rafael,-34.6177,-68.3301,25.88,14,0,3.14,AR,1697312911


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
115,gwadar,PK,25.1216,62.3254,85,
119,gremda,TN,34.7500,10.7833,65,
140,pacific grove,US,36.6177,-121.9166,62,
282,birzebbuga,MT,35.8258,14.5269,78,
292,paulden,US,34.8856,-112.4682,30,
329,metlili chaamba,DZ,32.2667,3.6333,21,
375,mckinleyville,US,40.9465,-124.1006,79,
396,laguna,US,38.4210,-121.4238,53,
419,teguise,ES,29.0605,-13.5640,61,
559,san rafael,AR,-34.6177,-68.3301,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey":geoapify_key,
          "categories":"accommodation.hotel"}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
gwadar - nearest hotel: Sadaf Resorts
gremda - nearest hotel: Business Hotel Sfax
pacific grove - nearest hotel: Pacific Grove Inn
birzebbuga - nearest hotel: Seabreeze
paulden - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
mckinleyville - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
teguise - nearest hotel: Finca Las Laderas
san rafael - nearest hotel: Hotel Regional


,City,Country,Lat,Lng,Humidity,Hotel Name
115,gwadar,PK,25.1216,62.3254,85,Sadaf Resorts
119,gremda,TN,34.7500,10.7833,65,Business Hotel Sfax
140,pacific grove,US,36.6177,-121.9166,62,Pacific Grove Inn
282,birzebbuga,MT,35.8258,14.5269,78,Seabreeze
292,paulden,US,34.8856,-112.4682,30,No hotel found
329,metlili chaamba,DZ,32.2667,3.6333,21,No hotel found
375,mckinleyville,US,40.9465,-124.1006,79,No hotel found
396,laguna,US,38.4210,-121.4238,53,Holiday Inn Express & Suites
419,teguise,ES,29.0605,-13.5640,61,Finca Las Laderas
559,san rafael,AR,-34.6177,-68.3301,14,Hotel Regional


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)